# DCGAN Training on Fashion-MNIST (Google Colab)

このノートブックでは、モジュール化されたDCGANコードを使用してFashion-MNISTデータセットで学習を行います。

## モジュール構成
- `config.py`: 設定とハイパーパラメータ
- `models.py`: Generator/Discriminator定義
- `data_loader.py`: データセット準備
- `train.py`: 学習ループ
- `visualize.py`: 可視化関数

## 0. Google Colabセットアップ（Colabでのみ実行）

In [ ]:
# Google Colabで実行する場合、GitHubからリポジトリをクローン
import os

# Colabかどうかを判定
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("Running in Google Colab")
    
    # リポジトリをクローン（YOUR_GITHUB_USERNAMEを実際のユーザー名に置き換えてください）
    if not os.path.exists('Pytorch-DCGAN'):
        !git clone https://github.com/YOUR_GITHUB_USERNAME/Pytorch-DCGAN.git
    
    # 作業ディレクトリを変更
    os.chdir('Pytorch-DCGAN')
    print(f"Current directory: {os.getcwd()}")
else:
    print("Running locally")

## 1. モジュールのインポート

In [ ]:
import sys
import os

# パスの設定（Colab/ローカル両対応）
if IN_COLAB:
    # Colabの場合: /content/Pytorch-DCGAN/src
    sys.path.insert(0, '/content/Pytorch-DCGAN/src')
else:
    # ローカルの場合: ../src
    sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

import torch
import torch.nn as nn

from config import Config
from models import Generator, Discriminator, weights_init
from data_loader import create_dataloader, visualize_batch
from train import DCGANTrainer
from visualize import plot_losses, create_animation, compare_real_fake

print("Modules imported successfully!")

## 2. 設定の初期化

In [ ]:
# ランダムシードを設定（再現性のため）
Config.set_seed()

# 使用するデバイスを取得
device = Config.get_device()
print(f"Using device: {device}")

# 設定の表示
print(f"\nConfiguration:")
print(f"  Batch Size: {Config.BATCH_SIZE}")
print(f"  Image Size: {Config.IMAGE_SIZE}")
print(f"  Num Epochs: {Config.NUM_EPOCHS}")
print(f"  Learning Rate: {Config.LR}")
print(f"  Latent Dim (nz): {Config.NZ}")

## 3. データセットの準備

In [ ]:
# DataLoaderの作成
dataloader = create_dataloader(
    dataroot=Config.DATAROOT,
    batch_size=Config.BATCH_SIZE,
    image_size=Config.IMAGE_SIZE,
    workers=Config.WORKERS
)

print(f"Dataset loaded: {len(dataloader)} batches")

## 4. 学習データの可視化

In [ ]:
# 学習データの一部を表示
visualize_batch(dataloader, num_images=64)

## 5. モデルの初期化

In [ ]:
# Generator（生成器）のインスタンスを作成
netG = Generator(
    ngpu=Config.NGPU,
    nz=Config.NZ,
    ngf=Config.NGF,
    nc=Config.NC
).to(device)

# GPUが利用可能かつ複数GPUを使用する場合
if (device.type == 'cuda') and (Config.NGPU > 1):
    netG = nn.DataParallel(netG, list(range(Config.NGPU)))

# 重みを初期化
netG.apply(weights_init)

print("Generator:")
print(netG)

In [ ]:
# Discriminator（識別器）のインスタンスを作成
netD = Discriminator(
    ngpu=Config.NGPU,
    ndf=Config.NDF,
    nc=Config.NC
).to(device)

# GPUが利用可能かつ複数GPUを使用する場合
if (device.type == 'cuda') and (Config.NGPU > 1):
    netD = nn.DataParallel(netD, list(range(Config.NGPU)))

# 重みを初期化
netD.apply(weights_init)

print("Discriminator:")
print(netD)

## 6. 学習の実行

In [ ]:
# トレーナーの初期化
trainer = DCGANTrainer(
    netG=netG,
    netD=netD,
    device=device,
    config=Config
)

# 学習の実行
trainer.train(dataloader, Config.NUM_EPOCHS)

## 7. 学習結果の取得

In [ ]:
# 学習結果を取得
results = trainer.get_results()
img_list = results['img_list']
G_losses = results['G_losses']
D_losses = results['D_losses']

print(f"Training completed!")
print(f"  Generated {len(img_list)} image snapshots")
print(f"  Recorded {len(G_losses)} loss values")

## 8. 損失推移の可視化

In [ ]:
# Generator / Discriminator の損失推移をプロット
plot_losses(G_losses, D_losses)

## 9. 生成画像のアニメーション

In [ ]:
# 学習過程での生成画像の変化をアニメーション表示
create_animation(img_list)

## 10. 本物 vs 生成画像の比較

In [ ]:
# 本物画像と生成画像を並べて表示
compare_real_fake(dataloader, img_list, device)